In [ ]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import pandas_profiling
import numpy as np
import re
import requests
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings
import types

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Read the task1_train data
train_df = pd.read_csv("drive/My Drive/task2_train.csv", low_memory=False)
train_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,17667,250.0,2013-02-07,Norad,1409,NORAD,29.936235,-4.422470e+00,Kwa Ntilio,0,Lake Tanganyika,Rusange,Kigoma,16,2,Kasulu,Janda,370,True,GeoData Consultants Ltd,Water authority,Nyafisi,True,1991,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,functional needs repair
1,73543,0.0,2013-10-03,Danida,436,DANIDA,34.607098,-1.055015e+01,Shuleni,0,Lake Nyasa,Mbozi,Ruvuma,10,3,Mbinga,Lituhi,160,False,GeoData Consultants Ltd,VWC,DANIDA,True,1994,gravity,gravity,gravity,parastatal,parastatal,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair
2,29657,0.0,2013-01-18,Dwsp,0,DWE,0.000000,-2.000000e-08,Mbugani,0,Lake Victoria,Mpilingito,Shinyanga,17,1,Bariadi,Dutwa,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,walimi,other handpump,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
3,12515,0.0,2013-02-03,Government Of Tanzania,991,DWE,30.394823,-5.106162e+00,Kwa Mzee Malundi,0,Lake Tanganyika,Lugongoni,Kigoma,16,3,Kigoma Rural,Uvinza,550,False,GeoData Consultants Ltd,VWC,Nyanza water project,True,1997,gravity,gravity,gravity,vwc,user-group,unknown,unknown,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe multiple,communal standpipe,functional needs repair
4,18756,0.0,2011-04-16,Kkkt,0,KKKT,33.002953,-9.064479e+00,Bahati Msanganzila,0,Lake Rukwa,Igawanya,Mbeya,12,6,Mbozi,Mlowo,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,other,other,other,vwc,user-group,pay when scheme fails,on failure,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,38745,20.0,2013-01-29,Finida German Tanzania Govt,506,Finwater,39.298712,-1.069067e+01,Kwa Mzee Chihembe,0,Ruvuma / Southern Coast,Manjenje,Mtwara,90,33,Newala,Malatu,1,True,GeoData Consultants Ltd,VWC,Mitema /MWS,False,1988,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional needs repair
6903,27918,500.0,2013-01-03,Government Of Tanzania,1227,RWE,36.908360,-3.356076e+00,Gad Palanje,0,Pangani,Lerai,Arusha,2,7,Meru,Maji ya Chai,120,True,GeoData Consultants Ltd,VWC,Tuvaila gravity water supply,True,1970,gravity,gravity,gravity,vwc,user-group,pay when scheme fails,on failure,soft,good,seasonal,seasonal,river,river/lake,surface,communal standpipe,communal standpipe,functional needs repair
6904,22550,0.0,2011-08-01,Tasaf,0,DWE,30.686126,-1.252510e+00,Kwaalphonce,0,Lake Victoria,Kigarama,Kagera,18,1,Karagwe,Kaisho,0,True,GeoData Consultants Ltd,WUG,Kaisho/Isingiro w,True,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair
6905,19770,0.0,2011-07-07,Hesawa,0,DWE,31.318268,-2.645071e+00,Kwa Lutema,0,Lake Tanganyika,Ng'Ambo,Kagera,18,4,Biharamulo,Biharamulo Urban,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump

In [ ]:
#Read the task2_test data
test_df = pd.read_csv("drive/My Drive/task2_test.csv", low_memory=False)
test_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,45786,0.0,2012-11-13,Biore,0,Biore,34.023367,-3.538302,Shinyanga Primary School,0,Internal,Mwambuluki,Shinyanga,17,2,Maswa,Dakama,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,gravity,gravity,gravity,wug,user-group,other,other,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional needs repair
1,39181,0.0,2013-02-03,Rotary Club,1141,Rotary club,36.901475,-3.391345,Kitefu Primary School,0,Pangani,Tanesco,Arusha,2,7,Meru,Maji ya Chai,360,True,GeoData Consultants Ltd,WUA,NaN,False,2000,gravity,gravity,gravity,wua,user-group,unknown,unknown,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,others
2,60073,0.0,2013-02-11,Ministry Of Water,1236,Idara ya maji,34.842991,-6.074815,Kanisani,0,Rufiji,Chikola,Singida,13,3,Manyoni,Chikola,300,False,GeoData Consultants Ltd,VWC,Chikola water supply,False,1975,mono,mono,motorpump,vwc,user-group,never pay,never pay,salty abandoned,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,others
3,10708,300.0,2013-02-26,Germany Republi,1327,CES,37.200640,-3.228018,Kwa Debora Mushi,0,Pangani,Bwani,Kilimanjaro,3,5,Hai,Machame Uroki,35,True,GeoData Consultants Ltd,Water Board,Uroki-Bomang'ombe water sup,True,1995,gravity,gravity,gravity,water board,user-group,other,other,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,others
4,60497,0.0,2011-07-06,Hewasa,0,DWE,31.517362,-1.601114,Shule Ya Msingi Nyakaju,0,Lake Victoria,Rubale,Kagera,18,2,Bukoba Rural,Rubale,0,True,GeoData Consultants Ltd,VWC,OLD RUBALE WATER SUPPLY SCHEME,True,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,40993,500.0,2011-11-04,Unicef,1428,DWE,34.597312,-8.947388,Kwa Mzee Mlengule,0,Rufiji,Kati,Iringa,11,4,Njombe,Usuka,250,True,GeoData Consultants Ltd,WUA,wanging'ombe water supply s,True,1984,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,functional needs repair
1723,52924,0.0,2011-03-15,World Bank,0,Word,36.552113,-6.687545,Chang'Ombe,0,Wami / Ruvu,Chang'Ombe,Dodoma,1,1,Mpwapwa,Lumuma,0,True,GeoData Consultants Ltd,VWC,Pwag,True,0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe multiple,communal standpipe,others
1724,55202,0.0,2012-10-22,Rwssp,0,DWE,33.517984,-3.430887,Tupendane,0,Internal,Bugeme,Shinyanga,17,8,Kishapu,Mondo,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,others
1725,2071,0.0,2013-03-21,Government Of Tanzania,1417,DWE,36.748423,-3.079985,Shule Ya Msingi,0,Internal,Engutukoit,Arusha,2,2,Arusha Rural,Oldonyosambu,150,True,GeoData Consultants Ltd,VWC,Losinoni pipe line,True,1960,gravity,gravity,gravity,vwc,user-group,never pay,never pay,fluoride,fluoride,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair


In [ ]:
train_df.shape

(6907, 41)

In [ ]:
train_df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [ ]:
train_df.isna().sum()

id                          0
amount_tsh                  0
date_recorded               0
funder                    567
gps_height                  0
installer                 570
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                 27
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting            307
recorded_by                 0
scheme_management         404
scheme_name              3200
permit                    405
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity  

In [ ]:
#Remove unwanted features
unwanted = ("id", "test", "train", "status_group", "num_private", "scheme_name",
           'waterpoint_type_group',
           'quality_group',
           'payment_type',
           'extraction_type_group',
           'extraction_type_class',
           'management_group',
           'source_type',
           'source_class')
#Select only features that do not start or match the unwanted ones
features = [col for col in list(train_df) if not col.startswith(unwanted)]

In [ ]:
features

['amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'permit',
 'construction_year',
 'extraction_type',
 'management',
 'payment',
 'water_quality',
 'quantity',
 'quantity_group',
 'source',
 'waterpoint_type']

In [ ]:
train_df[features].isna().sum()

amount_tsh             0
date_recorded          0
funder               567
gps_height             0
installer            570
longitude              0
latitude               0
wpt_name               0
basin                  0
subvillage            27
region                 0
region_code            0
district_code          0
lga                    0
ward                   0
population             0
public_meeting       307
recorded_by            0
scheme_management    404
permit               405
construction_year      0
extraction_type        0
management             0
payment                0
water_quality          0
quantity               0
quantity_group         0
source                 0
waterpoint_type        0
dtype: int64

In [ ]:
combined_data = pd.concat([train_df, test_df], ignore_index = True)
combined_data.head(5)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,17667,250.0,2013-02-07,Norad,1409,NORAD,29.936235,-4.422470e+00,Kwa Ntilio,0,Lake Tanganyika,Rusange,Kigoma,16,2,Kasulu,Janda,370,True,GeoData Consultants Ltd,Water authority,Nyafisi,True,1991,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,functional needs repair
1,73543,0.0,2013-10-03,Danida,436,DANIDA,34.607098,-1.055015e+01,Shuleni,0,Lake Nyasa,Mbozi,Ruvuma,10,3,Mbinga,Lituhi,160,False,GeoData Consultants Ltd,VWC,DANIDA,True,1994,gravity,gravity,gravity,parastatal,parastatal,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair
2,29657,0.0,2013-01-18,Dwsp,0,DWE,0.000000,-2.000000e-08,Mbugani,0,Lake Victoria,Mpilingito,Shinyanga,17,1,Bariadi,Dutwa,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,walimi,other handpump,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
3,12515,0.0,2013-02-03,Government Of Tanzania,991,DWE,30.394823,-5.106162e+00,Kwa Mzee Malundi,0,Lake Tanganyika,Lugongoni,Kigoma,16,3,Kigoma Rural,Uvinza,550,False,GeoData Consultants Ltd,VWC,Nyanza water project,True,1997,gravity,gravity,gravity,vwc,user-group,unknown,unknown,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe multiple,communal standpipe,functional needs repair
4,18756,0.0,2011-04-16,Kkkt,0,KKKT,33.002953,-9.064479e+00,Bahati Msanganzila,0,Lake Rukwa,Igawanya,Mbeya,12,6,Mbozi,Mlowo,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,other,other,other,vwc,user-group,pay when scheme fails,on failure,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,others


In [ ]:
#Remove unwanted features
unwanted = ("id", "test", "train", "status_group", "num_private", "scheme_name",
           'waterpoint_type_group',
           'quality_group',
           'payment_type',
           'extraction_type_group',
           'extraction_type_class',
           'management_group',
           'source_type',
           'source_class')
#Select only features that do not start or match the unwanted ones
features = [col for col in list(combined_data) if not col.startswith(unwanted)]

In [ ]:
combined_data[features].isna().sum()

amount_tsh             0
date_recorded          0
funder               705
gps_height             0
installer            711
longitude              0
latitude               0
wpt_name               0
basin                  0
subvillage            32
region                 0
region_code            0
district_code          0
lga                    0
ward                   0
population             0
public_meeting       379
recorded_by            0
scheme_management    494
permit               494
construction_year      0
extraction_type        0
management             0
payment                0
water_quality          0
quantity               0
quantity_group         0
source                 0
waterpoint_type        0
dtype: int64

Dealing with missing values

In [ ]:
# Shorten and combine the four featues
columns = [["installer", 3, 500],
        ["funder", 3, 50],
        ["subvillage", 4, 300],
        ["scheme_name", 5, 300]]
combine = "oth"

for column, chars, threshold in columns:
    # Missing values combined as a group
    combined_data.loc[combined_data[column].isnull() | combined_data[column].isin(["0", "-", "_"]), column] = column + "_" + combine
    # Shorten and convert to lowercase
    combined_data[column] = combined_data[column].map(lambda x: x[:chars].lower())
    # Combine datapoints which are lower than threshold
    combined_data.loc[combined_data[column].isin(combined_data[column].value_counts()[combined_data[column].value_counts() < threshold].index), column] = column + "_" + combine

In [ ]:
# preprocess latitude, longitude: round
cols = ["latitude",
        "longitude"]

for col in cols:
    combined_data[col] = combined_data[col].map(lambda x: round(x, 7))
    combined_data[col] = combined_data[col].replace(to_replace=-0.0, value=0)

combined_data[features].head(5)

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,quantity_group,source,waterpoint_type
0,250.0,2013-02-07,nor,1409,installer_oth,29.936235,-4.422470,Kwa Ntilio,Lake Tanganyika,subvillage_oth,Kigoma,16,2,Kasulu,Janda,370,True,GeoData Consultants Ltd,Water authority,True,1991,gravity,vwc,pay monthly,soft,insufficient,insufficient,river,communal standpipe
1,0.0,2013-10-03,dan,436,installer_oth,34.607098,-10.550154,Shuleni,Lake Nyasa,subvillage_oth,Ruvuma,10,3,Mbinga,Lituhi,160,False,GeoData Consultants Ltd,VWC,True,1994,gravity,parastatal,never pay,soft,enough,enough,spring,communal standpipe
2,0.0,2013-01-18,dws,0,dwe,0.000000,0.000000,Mbugani,Lake Victoria,subvillage_oth,Shinyanga,17,1,Bariadi,Dutwa,0,True,GeoData Consultants Ltd,WUG,False,0,walimi,wug,never pay,soft,enough,enough,shallow well,hand pump
3,0.0,2013-02-03,gov,991,dwe,30.394823,-5.106162,Kwa Mzee Malundi,Lake Tanganyika,subvillage_oth,Kigoma,16,3,Kigoma Rural,Uvinza,550,False,GeoData Consultants Ltd,VWC,True,1997,gravity,vwc,unknown,soft,insufficient,insufficient,river,communal standpipe multiple
4,0.0,2011-04-16,kkk,0,installer_oth,33.002953,-9.064479,Bahati Msanganzila,Lake Rukwa,subvillage_oth,Mbeya,12,6,Mbozi,Mlowo,0,True,GeoData Consultants Ltd,VWC,False,0,other,vwc,pay when scheme fails,soft,enough,enough,shallow well,other


In [ ]:
combined_data[features].shape

(8634, 29)

In [ ]:
combined_data[features].head(5)

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,quantity_group,source,waterpoint_type
0,250.0,2013-02-07,nor,1409,installer_oth,29.936235,-4.422470e+00,Kwa Ntilio,Lake Tanganyika,subvillage_oth,Kigoma,16,2,Kasulu,Janda,370,True,GeoData Consultants Ltd,Water authority,True,1991,gravity,vwc,pay monthly,soft,insufficient,insufficient,river,communal standpipe
1,0.0,2013-10-03,dan,436,installer_oth,34.607098,-1.055015e+01,Shuleni,Lake Nyasa,subvillage_oth,Ruvuma,10,3,Mbinga,Lituhi,160,False,GeoData Consultants Ltd,VWC,True,1994,gravity,parastatal,never pay,soft,enough,enough,spring,communal standpipe
2,0.0,2013-01-18,dws,0,dwe,0.000000,-2.000000e-08,Mbugani,Lake Victoria,subvillage_oth,Shinyanga,17,1,Bariadi,Dutwa,0,True,GeoData Consultants Ltd,WUG,False,0,walimi,wug,never pay,soft,enough,enough,shallow well,hand pump
3,0.0,2013-02-03,gov,991,dwe,30.394823,-5.106162e+00,Kwa Mzee Malundi,Lake Tanganyika,subvillage_oth,Kigoma,16,3,Kigoma Rural,Uvinza,550,False,GeoData Consultants Ltd,VWC,True,1997,gravity,vwc,unknown,soft,insufficient,insufficient,river,communal standpipe multiple
4,0.0,2011-04-16,kkk,0,installer_oth,33.002953,-9.064479e+00,Bahati Msanganzila,Lake Rukwa,subvillage_oth,Mbeya,12,6,Mbozi,Mlowo,0,True,GeoData Consultants Ltd,VWC,False,0,other,vwc,pay when scheme fails,soft,enough,enough,shallow well,other


In [ ]:
combined_data.columns

Index(['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'num_private', 'basin', 'subvillage', 'region',
       'region_code', 'district_code', 'lga', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [ ]:
#Drop constant features
drop = ("date_recorded",
        "recorded_by",
        "ward",
        "wpt_name")

#dynamic_df = combined_data.drop(drop, inplace=True)
clean1 = combined_data[features]
dropped = [col for col in list(clean1) if not col.startswith(drop)]
dynamic_df = clean1[dropped]
dynamic_df

KeyError: ignored

In [ ]:

from math import log
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
#import geocoder
#Round off the population and amount value fields
combined_data["amount_tsh"] = combined_data["amount_tsh"].apply(lambda x: log(round(x)) if round(x) > 0 else 0)
combined_data["population"] = combined_data["population"].apply(lambda x: log(x) if x > 0 else 0)

In [ ]:
combined_data

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,subvillage,region,region_code,district_code,lga,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,17667,5.521461,nor,1409,installer_oth,29.936235,-4.422470,0,Lake Tanganyika,subvillage_oth,Kigoma,16,2,Kasulu,5.913503,True,Water authority,scheme_name_oth,True,1991,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,functional needs repair
1,73543,0.000000,dan,436,installer_oth,34.607098,-10.550154,0,Lake Nyasa,subvillage_oth,Ruvuma,10,3,Mbinga,5.075174,False,VWC,scheme_name_oth,True,1994,gravity,gravity,gravity,parastatal,parastatal,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair
2,29657,0.000000,dws,0,dwe,0.000000,0.000000,0,Lake Victoria,subvillage_oth,Shinyanga,17,1,Bariadi,0.000000,True,WUG,schem,False,0,walimi,other handpump,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
3,12515,0.000000,gov,991,dwe,30.394823,-5.106162,0,Lake Tanganyika,subvillage_oth,Kigoma,16,3,Kigoma Rural,6.309918,False,VWC,scheme_name_oth,True,1997,gravity,gravity,gravity,vwc,user-group,unknown,unknown,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe multiple,communal standpipe,functional needs repair
4,18756,0.000000,kkk,0,installer_oth,33.002953,-9.064479,0,Lake Rukwa,subvillage_oth,Mbeya,12,6,Mbozi,0.000000,True,VWC,schem,False,0,other,other,other,vwc,user-group,pay when scheme fails,on failure,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8629,40993,6.214608,uni,1428,dwe,34.597312,-8.947388,0,Rufiji,subvillage_oth,Iringa,11,4,Njombe,5.521461,True,WUA,scheme_name_oth,True,1984,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,functional needs repair
8630,52924,0.000000,wor,0,installer_oth,36.552113,-6.687545,0,Wami / Ruvu,subvillage_oth,Dodoma,1,1,Mpwapwa,0.000000,True,VWC,scheme_name_oth,True,0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe multiple,communal standpipe,others
8631,55202,0.000000,rws,0,dwe,33.517984,-3.430887,0,Internal,subvillage_oth,Shinyanga,17,8,Kishapu,0.000000,True,WUG,schem,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,others
8632,2071,0.000000,gov,1417,dwe,36.748423,-3.079985,0,Internal,subvillage_oth,Arusha,2,2,Arusha Rural,5.010635,True,VWC,scheme_name_oth,True,1960,gravity,gravity,gravity,vwc,user-group,never pay,never pay,fluoride,fluoride,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair


In [ ]:
# Binary Classification with Sonar Dataset: Baseline
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
# load dataset
dataframe = data
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=60, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 84.45% (6.33%)
